In [1]:
!pip install XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip install dict

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for dict: filename=dict-2020.12.3-py3-none-any.whl size=1622 sha256=5eb6045d7c13cccb31dafdd8849ece115a215df9f2ebbd5196aaf65ffa3e34c7
  Stored in directory: /root/.cache/pip/wheels/be/16/91/2662873aebe20df5239ceddaa23e342c1619683ae231d1c9e3
  Created wheel for values: filename=values-2020.12.3-py3-none-any.whl size=1337 sha256=b163937fedf7648fe48f33f7ad93fddf3d9f5d15631f704beb83adf46ad98002
  Stored in directory: /root/.cache/pip/wheels/a7/bc/50/73bf2f3bc5ea44e3a0ca1ce38c2fa0d05967ef3ff5d5698978
Successfully built dict values


In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import dict

In [5]:
path = "/content/sp_500_stocks.csv"

In [6]:
stocks = pd.read_csv(path)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [7]:
symbol='AAPL'
api_url = f'https://ALGORITHMIC.iex.cloud/v1/data/core/quote/{symbol}?token=sk_604e2c9a98c947cc8048889f61234d07'
response = requests.get(api_url)
data = requests.get(api_url).json()
data

[{'avgTotalVolume': 73817592,
  'calculationPrice': 'iexlasttrade',
  'change': -0.96,
  'changePercent': -0.00562,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 169.89,
  'iexCloseTime': 1711483197650,
  'iexLastUpdated': 1711483197650,
  'iexMarketPercent': 0.011119387877592682,
  'iexOpen': 170.03,
  'iexOpenTime': 1711459800546,
  'iexRealtimePrice': 169.89,
  'iexRealtimeSize': 200,
  'iexVolume': 637480,
  'lastTradeTime': 1711483199393,
  'latestPrice': 169.89,
  'latestSource': 'IEX Last Trade',
  'latestTime': 'March 26, 2024',
  'latestUpdate': 1711483197650,
  'latestVolume': None,
  'low': None

In [8]:
data[0]['latestPrice']
data[0]['marketCap']

2623421163090

In [9]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [ ]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL',
                                                   data[0]['latestPrice'],
                                                   data[0]['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)
final_dataframe

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token=sk_604e2c9a98c947cc8048889f61234d07'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                   data[0]['latestPrice'],
                                                   data[0]['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)

In [14]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,144.410,42320113562,N/A
1,AAL,14.915,9765697911,N/A
2,AAP,84.250,5017175289,N/A
3,AAPL,169.890,2623421163090,N/A
4,ABBV,179.190,317282232884,N/A
...,...,...,...,...
500,YUM,134.800,37924130544,N/A
501,ZBH,127.870,26224093893,N/A
502,ZBRA,291.590,14976062400,N/A
503,ZION,41.220,6084031604,N/A


In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings [:i]:
    batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch?symbols={symbol_string}&types=quote&token=sk_604e2c9a98c947cc8048889f61234d07'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
            if symbol in data:
                symbol
                latest_price = data[symbol]['quote']['latestPrice']
                market_cap = data[symbol]['quote']['marketCap']
                final_dataframe = final_dataframe.append(
                    pd.Series([
                        symbol,
                        latest_price,
                        market_cap,
                        'N/A'
                    ],
                    index=my_columns),
                    ignore_index=True
                )
            else:
                print(f"No data found for symbol: {symbol}")

final_dataframe

In [23]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [24]:
position_size = val/ len(final_dataframe.index)
for i in range(0, len(final_dataframe.index) ):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,144.410,42320113562,14
1,AAL,14.915,9765697911,136
2,AAP,84.250,5017175289,24
3,AAPL,169.890,2623421163090,11
4,ABBV,179.190,317282232884,11
...,...,...,...,...
486,XLNX,194.920,48414620999,10
487,XOM,113.790,451501003694,17
488,XRAY,32.515,6742416919,62
489,XRX,17.680,2195548474,115


In [25]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)

0

In [28]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [29]:
writer.save()

<ipython-input-29-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
